In [1]:
# -*- coding:utf-8 -*-
# !/usr/bin/python
# Author: Selvaria

# 一小部分步幅部分和池化层的实现

import tensorflow as tf
import numpy as np
print(tf.__version__)

D:\Selvaria\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.0.0


In [3]:
int(4.5)

4

In [2]:
# 步幅的测试：
def comp_conv2d(conv2d, X):
    X = tf.reshape(X,(1,) + X.shape + (1,))
    Y = conv2d(X)
    #input_shape = (samples, rows, cols, channels)
    return tf.reshape(Y,Y.shape[1:3])

conv2d = tf.keras.layers.Conv2D(1, kernel_size=3, padding='same')
X = tf.random.uniform(shape=(8,8))
print(comp_conv2d(conv2d,X).shape)

X = tf.random.uniform(shape=(8,8))
conv2d_stride8 = tf.keras.layers.Conv2D(1, kernel_size=3, padding='same',strides=2)
print('input8*8:',comp_conv2d(conv2d_stride8, X).shape)

X = tf.random.uniform(shape=(9,9))
conv2d_stride9 = tf.keras.layers.Conv2D(1, kernel_size=3, padding='same',strides=2)
print('input9*9:',comp_conv2d(conv2d_stride9, X).shape)

(8, 8)
input8*8: (4, 4)
input9*9: (5, 5)


In [2]:
# 实现函数

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = tf.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w +1)) #这步和填充很像
    Y = tf.Variable(Y)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j].assign(tf.reduce_max(X[i:i+p_h, j:j+p_w]))
            elif mode =='avg':
                Y[i,j].assign(tf.reduce_mean(X[i:i+p_h, j:j+p_w]))
    return Y

X = tf.constant([[0,1,2],[3,4,5],[6,7,8]],dtype=tf.float32)
pool2d(X, (2,2))

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[4., 5.],
       [7., 8.]], dtype=float32)>

In [3]:
pool2d(X, (2,2), 'avg')

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [4]:
#input_shape = (samples, rows, cols, channels)
X = tf.reshape(tf.constant(range(16)), (1,4,4,1))
X

<tf.Tensor: id=117, shape=(1, 4, 4, 1), dtype=int32, numpy=
array([[[[ 0],
         [ 1],
         [ 2],
         [ 3]],

        [[ 4],
         [ 5],
         [ 6],
         [ 7]],

        [[ 8],
         [ 9],
         [10],
         [11]],

        [[12],
         [13],
         [14],
         [15]]]])>

In [6]:
#注意池化层的步幅默认和池化层的形状一样
pool2d = tf.keras.layers.MaxPool2D(pool_size=[3,3])  #默认的padding是valid，即不填充
pool2d(X)

<tf.Tensor: id=119, shape=(1, 1, 1, 1), dtype=int32, numpy=array([[[[10]]]])>

In [5]:
pool2d = tf.keras.layers.MaxPool2D(pool_size=[3,3],padding='same') #填充，似乎keras默认填充p=k-1，这里输出形状=(4-3+(3-1)+3)/3=2
pool2d(X)

<tf.Tensor: id=118, shape=(1, 2, 2, 1), dtype=int32, numpy=
array([[[[ 5],
         [ 7]],

        [[13],
         [15]]]])>

In [10]:
pool2d = tf.keras.layers.MaxPool2D(pool_size=[3,3],padding='same',strides=1) #填充使输入和输出的形状相同
# o = (4-3+(3-1)+1)/1=4
pool2d(X)

<tf.Tensor: id=118, shape=(1, 4, 4, 1), dtype=int32, numpy=
array([[[[ 5],
         [ 6],
         [ 7],
         [ 7]],

        [[ 9],
         [10],
         [11],
         [11]],

        [[13],
         [14],
         [15],
         [15]],

        [[13],
         [14],
         [15],
         [15]]]])>

In [11]:
# 在处理多通道输入数据时，池化层对每个输入通道分别池化，而不是像卷积层那样将各通道的输入按通道相加。这意味着池化层的输出通道数与输入通道数相等

X = tf.stack([X, X+1], axis=3)
X = tf.reshape(X, (2,4,4,1))
X.shape

TensorShape([2, 4, 4, 1])

In [12]:
pool2d = tf.keras.layers.MaxPool2D(3, padding='same', strides=2)
pool2d(X)

<tf.Tensor: id=124, shape=(2, 2, 2, 1), dtype=int32, numpy=
array([[[[ 5],
         [ 6]],

        [[ 7],
         [ 8]]],


       [[[13],
         [14]],

        [[15],
         [16]]]])>